<style>
    .center-image {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>


In [3]:
import ipywidgets as widgets
from IPython.display import display,clear_output, Image, HTML,Javascript
import joblib
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from Helperfunctions import clean_df
from Helperfunctions import generate_html_table
import threading


clean_data,leagues_dict,countries_dict,num_vars = clean_df()


out = widgets.Output()
# define function to generate predictions and ranges
def predict_player_ratings(_):
    
    with out:
        clear_output()
        # get selected values from dropdowns and text input

        if (
            not best_position_dropdown.value
            or not league_dropdown.value
            or not continent_dropdown.value
            or not nationality_dropdown.value
            or not last_season_dropdown.value
            or not footy_club_ranking.value
            or not transfermarkt_url_text.value
            or not besoccer_url_text.value
        ):
            display(Javascript('alert("Please fill in all the fields.")'))
            return

        # Validate Transfermarkt URL
        transfermarkt_url_pattern = r"^https://www\.transfermarkt\.com/.*?/profil/spieler/\d+$"
        if not re.match(transfermarkt_url_pattern, transfermarkt_url_text.value):
            display(Javascript('alert("Invalid Transfermarkt URL. Please enter a valid URL.")'))
            return

        # Validate Besoccer URL
        besoccer_url_pattern = r"^https://www\.besoccer\.com/player/.*$"
        if not re.match(besoccer_url_pattern, besoccer_url_text.value):
            display(Javascript('alert("Invalid Besoccer URL. Please enter a valid URL.")'))
            return

        # Disable predict button
        predict_button.disabled = True

        loading_bar = widgets.IntProgress(min=0, max=24, description='Loading:', bar_style='info')
        display(loading_bar)

        # Define function to update the loading bar
        def update_loading_bar():
            for i in range(25):
                loading_bar.value = i
                time.sleep(1)  # Sleep for 1 second to simulate progress

        # Start the loading bar update in a separate thread
        loading_thread = threading.Thread(target=update_loading_bar)
        loading_thread.start()

        best_position = best_position_dropdown.value
        league = league_dropdown.value
        continent = continent_dropdown.value
        nationality = nationality_dropdown.value
        club_ranking = footy_club_ranking.value
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136'
        }
        transfermarkt_url = transfermarkt_url_text.value
        time.sleep(1)
        response = requests.get(transfermarkt_url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')

        
        profil_image = soup.find("div",{"class":"data-header__profile-container"})

        image_url = profil_image.find("img").get("src")  # <-- Get the image URL from the text widget

        age_and_birthdate1 = soup.find('span',{"itemprop":"birthDate"})
        if age_and_birthdate1 is not None:
            try:
                age_and_birthdate = soup.find('span',{"itemprop":"birthDate"}).text.strip()
                birthday, age = age_and_birthdate.split('(')
                birthday = birthday.strip()
                age = int(age.replace(')', '').strip())
            except:
                print("There is no age in the transfermarket link")
                
                    # Get the player's market value from transfermarkt if theres no price just cotniue to the next
            market_value_tag = soup.find('span', {'class': 'waehrung'})
            if market_value_tag:
                value = market_value_tag.find_next_sibling(string=True).strip()
                multiplier = market_value_tag.find_next_sibling('span', {'class': 'waehrung'}).text.strip()
                if multiplier == 'm':
                    market_value = int(float(value) * 1000000)
                elif multiplier == 'k':
                        market_value = int(float(value) * 1000)
                else:
                    print("There is no transfermarkt value")
                    

        club = str(soup.find("span",{"class":"data-header__club"}).find('a').get("title"))

        url = transfermarkt_url.replace('profil', 'leistungsdatendetails')
        time.sleep(1)
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        best_position = best_position_dropdown.value
        if best_position != 'GK':
            tfoot = soup.select_one('tfoot')
        
            if tfoot is not None:
                tds = tfoot.select('td')
                club_apps = int(tds[4].text.replace('-', '0'))
                club_goals = int(tds[5].text.replace('-', '0'))
                club_assists = int(tds[6].text.replace('-', '0').replace('\xa0', '').replace('/', ''))
                club_mins = int(str(tds[8].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
            else:
                club_apps = 0
                club_goals = 0
                club_assists = 0
                club_mins = 0

            url = transfermarkt_url.replace('profil', 'nationalmannschaft')
            time.sleep(1)
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.content, 'html.parser')


            table = soup.find('table')
            if table is not None:
                tr = table.find('tr')
                if tr is not None:
                    country_name = table.find('a').get("title")

                    if 'U1'not in country_name.upper() and 'U2' not in country_name.upper():
                        tfoot = soup.find('tfoot')
                        if tfoot is not None:
                            tds = tfoot.find_all('td')
                            national_apps = int(tds[2].text.replace('-', '0'))
                            national_goals = int(tds[3].text.replace('-', '0'))
                            national_assists = int(tds[4].text.replace('-', '0'))
                            national_mins = int(str(tds[8].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
                        else:
                            national_apps = 0
                            national_goals = 0
                            national_assists = 0
                            national_mins = 0
                    else:
                        national_apps = 0
                        national_goals = 0
                        national_assists = 0
                        national_mins = 0
            else:
                    national_apps = 0
                    national_goals = 0
                    national_assists = 0
                    national_mins = 0
                
        else:
            tfoot = soup.select_one('tfoot')
        
            if tfoot is not None:
                tds = tfoot.select('td')
                club_apps = int(tds[4].text.replace('-', '0'))
                club_goals = int(tds[5].text.replace('-', '0'))
                club_assists = 0
                club_mins = int(str(tds[9].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))

            else:
                club_apps = 0
                club_goals = 0
                club_assists = 0
                club_mins = 0

            url = transfermarkt_url.replace('profil', 'nationalmannschaft')
            time.sleep(1)
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.content, 'html.parser')


            table = soup.find('table')
            if table is not None:
                tr = table.find('tr')
                if tr is not None:
                    country_name = table.find('a').get("title")
                    if 'U1'not in country_name.upper() and 'U2' not in country_name.upper():
                        tfoot = soup.find('tfoot')
                        if tfoot is not None:
                            tds = tfoot.find_all('td')
                            national_apps = int(tds[2].text.replace('-', '0'))
                            national_goals = int(tds[3].text.replace('-', '0'))
                            national_assists = 0
                            national_mins = int(str(tds[9].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))

                        else:
                            national_apps = 0
                            national_goals = 0
                            national_assists = 0
                            national_mins = 0
                    else:
                        national_apps = 0
                        national_goals = 0
                        national_assists = 0
                        national_mins = 0
            else:
                national_apps = 0
                national_goals = 0
                national_assists = 0
                national_mins = 0


        besoccer_link = besoccer_url_text.value
        time.sleep(1)
        response = requests.get(besoccer_link, headers=headers)


        # Parse the HTML response with BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the player's date of birth on the page
        elo_div = soup.find("div",{"class": "elo"})
        if elo_div is not None:
            elo = elo_div.text
            if elo == '-':
                bs_elo=25
            else:
                bs_elo= int(elo)

            main_div = soup.find("div",{"id":"mod_player_rating"})
            if main_div is not None:
                genral_ranking_table = main_div.find('div', {'class': 'panel-body table-list'})
                if genral_ranking_table is not None:
                    general_ranking_row = genral_ranking_table.find_all('div', {'class': 'table-row'})[1]
                    general_ranking_value = general_ranking_row.find_all('div')[1].text.strip('º') if general_ranking_row is not None else None

                    if  general_ranking_value is not None and general_ranking_value != '-':
                        bs_ranking = int(general_ranking_value)
                    else:
                        bs_ranking = 99999
                else:
                    bs_ranking = 99999

            else:
                bs_ranking = 99999
                
        else:
            bs_elo=25

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
        url = transfermarkt_url.replace('profil', 'erfolge')
        time.sleep(1)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        all_achievements = 0

        td_tags = soup.find_all('td', class_='hauptlink')
        if td_tags is not None:
            for td in td_tags:
                text = str(td.text.strip())
                match = re.match(r"\d+", text)
                number = int(match.group())
                all_achievements += int(number)
    
        else:
            all_achievements = 0

        url = transfermarkt_url.replace('profil', 'leistungsdatendetails')
        url = url + '/plus/0?saison=&verein=&liga=1&wettbewerb=&pos=&trainer_id='
        time.sleep(1)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        if best_position != 'GK':
            tfoot1 = soup.find('table',{"class":"items"})
        
            if tfoot1 is not None:
                tfoot = tfoot1.find('tfoot')
                tds = tfoot.select('td')
                if tds is not None:
                    club_apps_1st = int(tds[4].text.replace('-', '0'))
                    club_goals_1st = int(tds[5].text.replace('-', '0'))
                    club_assists_1st = int(tds[6].text.replace('-', '0').replace('\xa0', '').replace('/', ''))
                    club_mins_1st = int(str(tds[8].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
                else:
                    club_apps_1st = 0
                    club_goals_1st = 0
                    club_assists_1st = 0
                    club_mins_1st = 0
            else:
                club_apps_1st = 0
                club_goals_1st = 0
                club_assists_1st = 0
                club_mins_1st = 0
                
        else:
            tfoot1 = soup.find('table',{"class":"items"})
            
        
            if tfoot1 is not None:
                tfoot = tfoot1.find('tfoot')
                tds = tfoot.select('td')
                if tds is not None:
                    club_apps_1st = int(tds[4].text.replace('-', '0'))
                    club_goals_1st = int(tds[5].text.replace('-', '0'))
                    club_assists_1st = 0
                    club_mins_1st = int(str(tds[9].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
                else:
                    club_apps_1st = 0
                    club_goals_1st = 0
                    club_assists_1st = 0
                    club_mins_1st = 0

            else:
                club_apps_1st = 0
                club_goals_1st = 0
                club_assists_1st = 0
                club_mins_1st = 0


        url = transfermarkt_url.replace('profil', 'leistungsdaten')
        season_dict = {"2021/2022": "2021", "2022/2023": "2022"}
        season_text = last_season_dropdown.value
        if season_text in season_dict:
            season_text = season_dict[season_text] 
        url = url + '/plus/0?saison=' + season_text
        time.sleep(1)

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')


        if best_position != 'GK':
            tfoot1 = soup.find('table',{"class":"items"})
            
        
            if tfoot1 is not None:
                tfoot = tfoot1.find('tfoot')
                tds = tfoot.select('td')
                if tds is not None:
                    club_apps_last_season = int(tds[2].text.replace('-', '0'))
                    club_goals_last_season = int(tds[3].text.replace('-', '0'))
                    club_assists_last_season = int(tds[4].text.replace('-', '0').replace('\xa0', '').replace('/', ''))
                    club_mins_last_season = int(str(tds[8].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))

                else:
                    club_apps_last_season = 0
                    club_goals_last_season = 0
                    club_assists_last_season = 0
                    club_mins_last_season = 0

            else:
                club_apps_last_season = 0
                club_goals_last_season = 0
                club_assists_last_season = 0
                club_mins_last_season = 0
                
        else:
            tfoot1 = soup.find('table',{"class":"items"})
            

            if tfoot1 is not None:
                tfoot = tfoot1.find('tfoot')
                tds = tfoot.select('td')
                if tds is not None:
                    club_apps_last_season = int(tds[2].text.replace('-', '0'))
                    club_goals_last_season = int(tds[3].text.replace('-', '0'))
                    club_assists_last_season = 0
                    club_mins_last_season = int(str(tds[9].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))

                else:
                    club_apps_last_season = 0
                    club_goals_last_season = 0
                    club_assists_last_season = 0
                    club_mins_last_season = 0                    
            else:
                club_apps_last_season = 0
                club_goals_last_season = 0
                club_assists_last_season = 0
                club_mins_last_season = 0
        time.sleep(1)
#------------------------------------------------------------------------------------------------------------------------------
        url = transfermarkt_url.replace('profil', 'detaillierteleistungsdaten')
        session = requests.session()
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136'}
        session.headers.update(headers)
        time.sleep(1)            
        response = session.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tfoot1 = soup.find_all('table', {"class": "items"})

        if tfoot1:    
            last_table = tfoot1[-1]

            table_name = str(last_table.find_parent('div').find_parent('div').find_parent('div').find('h2').text)


            if table_name == "International cups":
                tfoot = last_table.find('tfoot')
                
                tds = tfoot.select('td')
                if best_position != 'GK':
                    if tds is not None:
                            club_apps_int_cups = int(tds[4].text.replace('-', '0'))
                            club_goals_int_cups = int(tds[5].text.replace('-', '0'))
                            club_assists_int_cups = int(tds[6].text.replace('-', '0').replace('\xa0', '').replace('/', ''))
                            club_mins_int_cups = int(str(tds[10].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))

                            body = last_table.find('tbody')
                            competitions = body.find_all('td',{"class":"hauptlink no-border-links"})
                            # Initialize lists to hold the aggregated data for each competition
                            comp_names = ['CAF-Champions League', 'CONCACAF Champions League', 'Libertadores', 'Copa Sudamericana', 'Champions League Qu.',
                                          'Champions League', 'Europa League', 'Europa League Qualifying', 'Conference League', 'UECL Qualifiers',
                                          'UEFA Youth League', 'AFC Champions League']
                            comp_apps = [0] * len(comp_names)
                            comp_goals = [0] * len(comp_names)
                            comp_assists = [0] * len(comp_names)
                            comp_mins = [0] * len(comp_names)                                
                            # Iterate through the 'competitions' list and aggregate the parsed values for each competition
                            for comp in competitions:
                                title = comp.find('a').get('title')
                                if title in comp_names:
                                    index = comp_names.index(title)
                                    
                                    club_apps_comp = int(comp.find_next('td').find_next('td').text.replace('-', '0'))
                                    
                                    club_goals_comp = int(comp.find_next('td').find_next('td').find_next('td').text.replace('-', '0'))
                                    
                                    club_assists_comp = int(comp.find_next('td').find_next('td').find_next('td').find_next('td').text.replace('-', '0').replace('\xa0', '').replace('/', ''))
                                    
                                    club_mins_comp = int(str(comp.find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
                                    comp_apps[index] += club_apps_comp
                                    comp_goals[index] += club_goals_comp
                                    comp_assists[index] += club_assists_comp
                                    comp_mins[index] += club_mins_comp

                            comp_data = {}

                            # Add the aggregated data as new columns to the dataframe
                            for ind, comp_name in enumerate(comp_names):
                                comp_data[f"{comp_name}_apps"] = comp_apps[ind]
                                comp_data[f"{comp_name}_goals"] = comp_goals[ind]
                                comp_data[f"{comp_name}_assists"] = comp_assists[ind]
                                comp_data[f"{comp_name}_mins"] = comp_mins[ind]

                else:
                    if tds is not None:
                        club_apps_int_cups = int(tds[4].text.replace('-', '0'))
                        club_goals_int_cups = int(tds[5].text.replace('-', '0'))
                        club_assists_int_cups = 0
                        club_mins_int_cups = int(str(tds[11].text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))


                        body = last_table.find('tbody')
                        competitions = body.find_all('td',{"class":"hauptlink no-border-links"})
                        # Initialize lists to hold the aggregated data for each competition
                        comp_names = ['CAF-Champions League', 'CONCACAF Champions League', 'Libertadores', 'Copa Sudamericana', 'Champions League Qu.',
                                      'Champions League', 'Europa League', 'Europa League Qualifying', 'Conference League', 'UECL Qualifiers',
                                      'UEFA Youth League', 'AFC Champions League']
                        comp_apps = [0] * len(comp_names)
                        comp_goals = [0] * len(comp_names)
                        comp_assists = [0] * len(comp_names)
                        comp_mins = [0] * len(comp_names)                                
                        # Iterate through the 'competitions' list and aggregate the parsed values for each competition
                        for comp in competitions:
                            title = comp.find('a').get('title')
                            if title in comp_names:
                                index = comp_names.index(title)
                                club_apps_comp = int(comp.find_next('td').find_next('td').text.replace('-', '0'))
                                club_goals_comp = int(comp.find_next('td').find_next('td').find_next('td').text.replace('-', '0'))
                                
                                club_assists_comp = 0
                                club_mins_comp = int(str(comp.find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.replace('-', '0').replace("'", '').replace('\xa0', '')).replace('.', ''))
                                comp_apps[index] += club_apps_comp
                                comp_goals[index] += club_goals_comp
                                comp_assists[index] += club_assists_comp
                                comp_mins[index] += club_mins_comp

                        # Add the aggregated data as new columns to the dataframe
                        comp_data = {}

                        for ind, comp_name in enumerate(comp_names):
                            comp_data[f"{comp_name}_apps"] = comp_apps[ind]
                            comp_data[f"{comp_name}_goals"] = comp_goals[ind]
                            comp_data[f"{comp_name}_assists"] = comp_assists[ind]
                            comp_data[f"{comp_name}_mins"] = comp_mins[ind]

            else:
                club_apps_int_cups = 0
                club_goals_int_cups = 0
                club_assists_int_cups = 0
                club_mins_int_cups = 0
                comp_names = ['CAF-Champions League', 'CONCACAF Champions League', 'Libertadores', 'Copa Sudamericana', 'Champions League Qu.',
                              'Champions League', 'Europa League', 'Europa League Qualifying', 'Conference League', 'UECL Qualifiers',
                              'UEFA Youth League', 'AFC Champions League']
                comp_apps = [0] * len(comp_names)
                comp_goals = [0] * len(comp_names)
                comp_assists = [0] * len(comp_names)
                comp_mins = [0] * len(comp_names)  
                comp_data = {}
                for ind, comp_name in enumerate(comp_names):
                    comp_data[f"{comp_name}_apps"] = comp_apps[ind]
                    comp_data[f"{comp_name}_goals"] = comp_goals[ind]
                    comp_data[f"{comp_name}_assists"] = comp_assists[ind]
                    comp_data[f"{comp_name}_mins"] = comp_mins[ind]

        else:
            club_apps_int_cups = 0
            club_goals_int_cups = 0
            club_assists_int_cups = 0
            club_mins_int_cups = 0
            comp_names = ['CAF-Champions League', 'CONCACAF Champions League', 'Libertadores', 'Copa Sudamericana', 'Champions League Qu.', 'Champions League',
                          'Europa League', 'Europa League Qualifying', 'Conference League', 'UECL Qualifiers', 'UEFA Youth League', 'AFC Champions League']
            comp_apps = [0] * len(comp_names)
            comp_goals = [0] * len(comp_names)
            comp_assists = [0] * len(comp_names)
            comp_mins = [0] * len(comp_names)
            comp_data = {}
            for ind, comp_name in enumerate(comp_names):
                comp_data[f"{comp_name}_apps"] = comp_apps[ind]
                comp_data[f"{comp_name}_goals"] = comp_goals[ind]
                comp_data[f"{comp_name}_assists"] = comp_assists[ind]
                comp_data[f"{comp_name}_mins"] = comp_mins[ind]




        image = Image(url=image_url)  # <-- Create an Image object
        html_string = f'<div style="display: flex; justify-content: center;"><img src="{image_url}" style="display: block; margin: 0 auto;"></div>'

        # Display the centered image using HTML
        display(HTML(html_string))

        # create new data frame with the user input
        new_data = pd.DataFrame({
            'age': [age],
            f'league_name_{league}': [1],
            f'best_position_{best_position}': [1],
            f'continent_{continent}': [1],
            f'nationality_{nationality}': [1],
            'market_value': [market_value],
            'Club appearances' : [club_apps],
            'Club Goals' :[club_goals],
            'Club Assists':[club_assists],
            'Club Minutes':[club_mins],
            'National appearances':[national_apps],
            'National Goals':[national_goals],
            'National Assists':[national_assists],
            'National Minutes':[national_mins],
            'Club Ranking footysimulator':[club_ranking],
            'besoccer_player_Ranking':[bs_ranking],
            'bessocer_elo':[bs_elo],
            'achievemnts_count' : [all_achievements],
            'First Tier appearances':[club_apps_1st],
            'First Tier Goals':[club_goals_1st],
            'First Tier Assists':[club_assists_1st],
            'First Tier Minutes':[club_mins_1st],
            'Last season appearances':[club_apps_last_season],
            'Last season Goals':[club_goals_last_season],
            'Last season Assists':[club_assists_last_season],
            'Last season Minutes':[club_mins_last_season],
            'int_cups_appearances':[club_apps_int_cups],
            'int_cups_goals':[club_goals_int_cups],
            'int_cups_assists':[club_assists_int_cups],
            'int_cups_minutes':[club_mins_int_cups],
            'CAF-Champions League_apps':[comp_data['CAF-Champions League_apps']],	
            'CAF-Champions League_goals':[comp_data['CAF-Champions League_goals']],	
            'CAF-Champions League_assists':[comp_data['CAF-Champions League_assists']],	
            'CAF-Champions League_mins':[comp_data['CAF-Champions League_mins']],	
            'CONCACAF Champions League_apps':[comp_data['CONCACAF Champions League_apps']],	
            'CONCACAF Champions League_goals':[comp_data['CONCACAF Champions League_goals']],	
            'CONCACAF Champions League_assists':[comp_data['CONCACAF Champions League_assists']],	
            'CONCACAF Champions League_mins':[comp_data['CONCACAF Champions League_mins']],	
            'Libertadores_apps':[comp_data['Libertadores_apps']],
            'Libertadores_goals':[comp_data['Libertadores_goals']],	
            'Libertadores_assists':[comp_data['Libertadores_assists']],	
            'Libertadores_mins':[comp_data['Libertadores_mins']],	
            'Copa Sudamericana_apps':[comp_data['Copa Sudamericana_apps']],	
            'Copa Sudamericana_goals':[comp_data['Copa Sudamericana_goals']],	
            'Copa Sudamericana_assists':[comp_data['Copa Sudamericana_assists']],	
            'Copa Sudamericana_mins':[comp_data['Copa Sudamericana_mins']],	
            'Champions League Qu._apps':[comp_data['Champions League Qu._apps']],	
            'Champions League Qu._goals':[comp_data['Champions League Qu._goals']],	
            'Champions League Qu._assists':[comp_data['Champions League Qu._assists']],	
            'Champions League Qu._mins':[comp_data['Champions League Qu._mins']],	
            'Champions League_apps':[comp_data['Champions League_apps']],	
            'Champions League_goals':[comp_data['Champions League_goals']],	
            'Champions League_assists':[comp_data['Champions League_assists']],	
            'Champions League_mins':[comp_data['Champions League_mins']],	
            'Europa League_apps':[comp_data['Europa League_apps']],	
            'Europa League_goals':[comp_data['Europa League_goals']],	
            'Europa League_assists':[comp_data['Europa League_assists']],	
            'Europa League_mins':[comp_data['Europa League_mins']],	
            'Europa League Qualifying_apps':[comp_data['Europa League Qualifying_apps']],	
            'Europa League Qualifying_goals':[comp_data['Europa League Qualifying_goals']],	
            'Europa League Qualifying_assists':[comp_data['Europa League Qualifying_assists']],	
            'Europa League Qualifying_mins':[comp_data['Europa League Qualifying_mins']],	
            'Conference League_apps':[comp_data['Conference League_apps']],	
            'Conference League_goals':[comp_data['Conference League_goals']],	
            'Conference League_assists':[comp_data['Conference League_assists']],	
            'Conference League_mins':[comp_data['Conference League_mins']],	
            'UECL Qualifiers_apps':[comp_data['UECL Qualifiers_apps']],	
            'UECL Qualifiers_goals':[comp_data['UECL Qualifiers_goals']],	
            'UECL Qualifiers_assists':[comp_data['UECL Qualifiers_assists']],	
            'UECL Qualifiers_mins':[comp_data['UECL Qualifiers_mins']],	
            'UEFA Youth League_apps':[comp_data['UEFA Youth League_apps']],	
            'UEFA Youth League_goals':[comp_data['UEFA Youth League_goals']],	
            'UEFA Youth League_assists':[comp_data['UEFA Youth League_assists']],	
            'UEFA Youth League_mins':[comp_data['UEFA Youth League_mins']],	
            'AFC Champions League_apps':[comp_data['AFC Champions League_apps']],	
            'AFC Champions League_goals':[comp_data['AFC Champions League_goals']],	
            'AFC Champions League_assists':[comp_data['AFC Champions League_assists']],	
            'AFC Champions League_mins':[comp_data['AFC Champions League_mins']],
        })
        
        # load the trained CatBoostRegressor models
        model_overall = joblib.load('CatBoostRegressor_overall.pk1')
        model_potential = joblib.load('CatBoostRegressor_potential.pk1')
        scaler = joblib.load('scaler.pk1')
        
        # get a list of missing columns in new_data
        # missing_cols = list(set(clean_data1.columns) - set(new_data.columns))
        missing_cols = list(set(model_overall.feature_names_) - set(new_data.columns))

        # add missing columns to new_data and set their values to 0
        # new_data = new_data.reindex(columns=list(new_data.columns) + missing_cols, fill_value=0)
        new_data = new_data.reindex(columns=list(new_data.columns) + missing_cols, fill_value=0)


        # scale numerical variables using the same StandardScaler object used for training
        new_data[num_vars] = scaler.transform(new_data[num_vars])


        pred_overall = model_overall.predict(new_data)
        pred_potential = model_potential.predict(new_data)

        html_table = generate_html_table(age,league,club,comp_data,best_position,nationality,market_value,all_achievements,club_apps_int_cups,club_goals_int_cups,club_assists_int_cups,club_mins_int_cups,
                                         club_goals,national_goals,club_apps,national_apps,club_mins,national_mins,club_assists,
                                         national_assists,pred_overall,pred_potential)
        loading_thread.join()

        loading_bar.close()

        display(HTML(html_table))
        
        predict_button.disabled = False



# define dropdown widgets and text input
leagues = sorted([col.split('_')[2] for col in clean_data.columns if 'league_name_' in col])
best_positions = sorted([col.split('_')[2] for col in clean_data.columns if 'best_position_' in col])
continents = sorted([col.split('_')[1] for col in clean_data.columns if 'continent_' in col])
nationalities = sorted([col.split('_')[1] for col in clean_data.columns if 'nationality_' in col])
clubs = sorted([col.split('_')[2] for col in clean_data.columns if 'club_name_' in col])
years = ['2021/2022','2022/2023']

league_dropdown = widgets.Dropdown(options=sorted(leagues), description='League:')
best_position_dropdown = widgets.Dropdown(options=sorted(best_positions), description='Best Position:')
continent_dropdown = widgets.Dropdown(options=sorted(continents), description='Continent:')
nationality_dropdown = widgets.Dropdown(options=sorted(nationalities), description='Nationality:')
transfermarkt_url_text = widgets.Text(description='Transfermarkt link:', style={'description_width': 'initial'})
besoccer_url_text = widgets.Text(description='Besoccer link:')
footy_club_ranking =  widgets.FloatText(description='Footy club ranking:', style={'description_width': 'initial'})
last_season_dropdown = widgets.Dropdown(options=years, description='Last Season:')
# define function to update nationality dropdown options based on selected continent
def update_nationalities(*args):
    selected_continent = continent_dropdown.value
    nationalities = countries_dict[selected_continent]
    nationality_dropdown.options = sorted(nationalities)


continent_dropdown.observe(update_nationalities, 'value')

predict_button = widgets.Button(description='Predict', button_style='success', style={'description_width': 'initial'})
predict_button.on_click(predict_player_ratings)



display(best_position_dropdown)
display(league_dropdown)
display(continent_dropdown)
display(nationality_dropdown)
display(last_season_dropdown)
display(footy_club_ranking)
display(transfermarkt_url_text)
display(besoccer_url_text)
display(predict_button)
display(out)

Dropdown(description='Best Position:', options=('CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',…

Dropdown(description='League:', options=('Free Agent', '[Argentina 1st Tier] Liga Profesional', '[Argentina 2n…

Dropdown(description='Continent:', options=('Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South Ame…

Dropdown(description='Nationality:', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antig…

Dropdown(description='Last Season:', options=('2021/2022', '2022/2023'), value='2021/2022')

FloatText(value=0.0, description='Footy club ranking:', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Transfermarkt link:', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Besoccer link:')

Button(button_style='success', description='Predict', style=ButtonStyle())

Output()

<style>
    .center-image {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>


<style>
    .center-image {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>


<style>
    .center-image {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>


<style>
    .center-image {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>
